### Intégration de PostgreSQL avec Elasticsearch

Elasticsearch permet :  
- ✅ Recherche full-text ultra-rapide
- ✅ Recherche approximative, tolérance aux fautes de frappe
- ✅ Agrégation et filtrage avancés

### Synchronisation des données : 3 méthodes principales

### Synchronisation des données : 3 méthodes principales

|Méthode|Avantages|Inconvénients|
|:---|:---|:---|
|Logstash + JDBC|Facile à mettre en place|Pas en temps réel|
|Debezium + Kafka|Temps réel via CDC (Change Data Capture)|Configuration complexe|
|Python + elasticsearch-py|Flexible et personnalisable|Code supplémentaire à gérer|


### Synchronisation avec un service Python (elasticsearch-py)

- Installer les bibliothèques nécessaires

```bash
pip install psycopg2 elasticsearch
```

- Script Python pour insérer les données dans Elasticsearch

```python
import psycopg2
from elasticsearch import Elasticsearch

# Connexion à PostgreSQL
conn = psycopg2.connect(
    dbname="mabasedb", user="monutilisateur", password="monmotdepasse", host="localhost"
)
cur = conn.cursor()

# Connexion à Elasticsearch
es = Elasticsearch(["http://localhost:9200"])

# Récupérer les données
cur.execute("SELECT id, nom, description FROM produits")
for row in cur.fetchall():
    doc = {
        "id": row[0],
        "nom": row[1],
        "description": row[2],
    }
    es.index(index="produits", id=row[0], body=doc)

print("Données insérées dans Elasticsearch !")

# Fermer les connexions
cur.close()
conn.close()
```

### Utilisation hybride PostgreSQL + Elasticsearch

Exemple :  
- 1️⃣ PostgreSQL stocke les produits avec des relations
- 2️⃣ Elasticsearch indexe uniquement les colonnes utiles pour la recherche
- 3️⃣ L’application interroge Elasticsearch pour la recherche, puis récupère les détails dans PostgreSQL

```python
# Recherche full-text dans Elasticsearch
search_query = "smartphone"
response = es.search(index="produits", body={
    "query": {
        "match": {"description": search_query}
    }
})

# Récupération des IDs depuis Elasticsearch
produit_ids = [hit["_id"] for hit in response["hits"]["hits"]]

# Requête PostgreSQL pour récupérer les détails des produits
cur.execute("SELECT * FROM produits WHERE id = ANY(%s)", (produit_ids,))
produits = cur.fetchall()
```